In [5]:
from neo4j import GraphDatabase

In [6]:
URI = "neo4j+s://398500ef.databases.neo4j.io"
AUTH = ("neo4j", "l_1gCVeywnEgHdmLxf93BdT7kp01G30o5ZpTORaI-y4")

In [7]:
driver = GraphDatabase.driver(URI, auth=(AUTH))

In [3]:
def run_query(query, parameters=None):
    with driver.session() as session:
        results = session.run(query, parameters)
        return [dict(record) for record in results]

In [15]:
def create_user(name, age, location, interests):
    query = """
            CREATE (u:User {name: $name, age: $age, location: $location, interests: $interests})
            RETURN u
            """
    parameters = {"name": name, "age": age, "location":location, "interests":interests}
    result = run_query(query, parameters)
    print(result)

[{'u': <Node element_id='4:73dec725-ccdf-437b-bfd5-623161b3662c:42' labels=frozenset({'User'}) properties={'name': 'Charlie', 'location': 'New York', 'interests': ['Music', 'Running'], 'age': 22}>}]


In [17]:
def update_user(name, **kwargs):
    set = ",".join([f"u.{key} = ${key}" for key in kwargs])
    query = ("MATCH (u:User {name: $name}) "
            f"SET {set} "
            "RETURN u")
            
    parameters = {"name": name, **kwargs}
    result = run_query(query, parameters)
    print(result)

[{'u': <Node element_id='4:73dec725-ccdf-437b-bfd5-623161b3662c:20' labels=frozenset({'User'}) properties={'name': 'Bob', 'location': 'California', 'interests': ['movies', 'reading'], 'age': 25}>}]


[{'u1': <Node element_id='4:73dec725-ccdf-437b-bfd5-623161b3662c:20' labels=frozenset({'User'}) properties={'name': 'Bob', 'location': 'California', 'interests': ['movies', 'reading'], 'age': 25}>, 'u2': <Node element_id='4:73dec725-ccdf-437b-bfd5-623161b3662c:42' labels=frozenset({'User'}) properties={'name': 'Charlie', 'location': 'New York', 'interests': ['Music', 'Running'], 'age': 22}>}]


[]


[{'u1': <Node element_id='4:73dec725-ccdf-437b-bfd5-623161b3662c:19' labels=frozenset({'User'}) properties={'name': 'Alice', 'location': 'New York', 'interests': ['Music', 'Running'], 'age': 32}>, 'u2': <Node element_id='4:73dec725-ccdf-437b-bfd5-623161b3662c:20' labels=frozenset({'User'}) properties={'name': 'Bob', 'location': 'California', 'interests': ['movies', 'reading'], 'age': 25}>}]


[]


[{'p': <Node element_id='4:73dec725-ccdf-437b-bfd5-623161b3662c:22' labels=frozenset({'Post'}) properties={'content': 'try again'}>}]


[{'p': <Node element_id='4:73dec725-ccdf-437b-bfd5-623161b3662c:22' labels=frozenset({'Post'}) properties={'content': 'try again'}>}]


[{'g': <Node element_id='4:73dec725-ccdf-437b-bfd5-623161b3662c:41' labels=frozenset({'Group'}) properties={'name': 'admin'}>}]


[{'u': <Node element_id='4:73dec725-ccdf-437b-bfd5-623161b3662c:19' labels=frozenset({'User'}) properties={'name': 'Alice', 'location': 'New York', 'interests': ['Music', 'Running'], 'age': 32}>, 'g': <Node element_id='4:73dec725-ccdf-437b-bfd5-623161b3662c:41' labels=frozenset({'Group'}) properties={'name': 'admin'}>}]


[{'p': <Node element_id='4:73dec725-ccdf-437b-bfd5-623161b3662c:22' labels=frozenset({'Post'}) properties={'content': 'try again'}>}]


[{'ru': <Node element_id='4:73dec725-ccdf-437b-bfd5-623161b3662c:42' labels=frozenset({'User'}) properties={'name': 'Charlie', 'location': 'New York', 'interests': ['Music', 'Running'], 'age': 22}>}]


[{'u': <Node element_id='4:73dec725-ccdf-437b-bfd5-623161b3662c:19' labels=frozenset({'User'}) properties={'name': 'Alice', 'location': 'New York', 'interests': ['Music', 'Running'], 'age': 32}>}]
